In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import xgboost as xgb

In [2]:
#load cleaned dataset
df= pd.read_parquet("cleaned_data.parquet")
df.head()

,ffb_1%_oer,import,export,production,end_stock,cpo_futures,usd_myr_rate,brent_oil_futures,soybean_futures,precipitation,...,avg_humidity,lag_1,lag_3,lag_7,rolling_mean_7,rolling_mean_30,rolling_std_7,rolling_std_30,pct_change_1,pct_change_7
0,21.2,81477,1680891,1737461,3002871,2204.0,4.1075,61.65,30.74,20.6,...,88.041667,21.25,20.85,20.60,20.992857,20.533333,0.212972,0.319032,-0.002353,0.029126
1,21.3,81477,1680891,1737461,3002871,2200.0,4.0960,61.89,30.48,47.5,...,90.083333,21.20,21.20,20.75,21.071429,20.576667,0.209875,0.332113,0.004717,0.026506
2,21.3,94278,1324615,1544518,3056929,2200.0,4.0960,62.75,30.21,7.0,...,89.958333,21.30,21.25,20.85,21.135714,20.620000,0.199404,0.339015,0.000000,0.021583
3,21.3,94278,1324615,1544518,3056929,2200.0,4.0960,62.75,30.21,4.7,...,90.083333,21.30,21.20,20.85,21.200000,20.660000,0.160728,0.346261,0.000000,0.021583
4,21.3,94278,1324615,1544518,3056929,2200.0,4.0960,62.75,30.21,13.2,...,89.125000,21.30,21.30,20.85,21.264286,20.690000,0.047559,0.361606,0.000000,0.021583


In [3]:
target_col = "ffb_1%_oer"
raw_features = ["import", "export", "production", "end_stock", 
                "cpo_futures", "usd_myr_rate", "brent_oil_futures", 
                "soybean_futures", "precipitation", "avg_temperature", "avg_humidity"]

engineered_features = ["lag_1","lag_3","lag_7","rolling_mean_7",
                       "rolling_mean_30","rolling_std_7","rolling_std_30",
                       "pct_change_1","pct_change_7"]

engineered_features_lstm = ["lag_1", "rolling_mean_7"]

X = df[raw_features + engineered_features].values
y = df[target_col].values.reshape(-1,1)


#Splitting into train-validate-test dataa
N = len(df)
train_size = int(N * 0.7)   # 70% train
val_size   = int(N * 0.2)  # 20% validation
test_size  = N - train_size - val_size  # 10% test

X_train_raw = X[:train_size]
X_val_raw   = X[train_size:train_size+val_size]
X_test_raw  = X[train_size+val_size:]

y_train_raw = y[:train_size]
y_val_raw   = y[train_size:train_size+val_size]
y_test_raw  = y[train_size+val_size:]


#scale data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_x.fit_transform(X_train_raw)
X_val   = scaler_x.transform(X_val_raw)
X_test  = scaler_x.transform(X_test_raw)

y_train = scaler_y.fit_transform(y_train_raw)
y_val   = scaler_y.transform(y_val_raw)
y_test  = scaler_y.transform(y_test_raw)

In [4]:
def create_multi_step_sequences(X, y, lookback, horizon):
    Xs, ys = [], []
    for i in range(lookback, len(X) - horizon + 1):
        Xs.append(X[i - lookback:i])
        ys.append(y[i:i + horizon].ravel())  # collect next horizon steps
    return np.array(Xs), np.array(ys)

forecast_horizon = 7
lookback = 90
X_train_lstm, y_train_lstm = create_multi_step_sequences(X_train, y_train, lookback, forecast_horizon)
X_val_lstm, y_val_lstm     = create_multi_step_sequences(X_val, y_val, lookback, forecast_horizon)
X_test_lstm, y_test_lstm   = create_multi_step_sequences(X_test, y_test, lookback, forecast_horizon)


In [5]:
from tensorflow.keras import backend as K
import tensorflow as tf
import random
from tensorflow.keras.callbacks import EarlyStopping

#Clear previous model
K.clear_session()

#fix random seeds for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(lookback, X_train.shape[1])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(forecast_horizon)  # output 7 values simultaneously
])

model.compile(optimizer="adam", loss="mse")

#EarlyStopping Callback
early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 15,
    restore_best_weights=True
)

history = model.fit(
    X_train_lstm, y_train_lstm,
    validation_data=(X_val_lstm, y_val_lstm),
    epochs = 50,
    batch_size = 32,
    callbacks = [early_stop],
    verbose=1
)

y_pred_lstm = model.predict(X_test_lstm)

# Shape is now (samples, horizon)
y_pred_lstm_inv = scaler_y.inverse_transform(y_pred_lstm.reshape(-1,1)).reshape(y_pred_lstm.shape)
y_test_lstm_inv = scaler_y.inverse_transform(y_test_lstm.reshape(-1,1)).reshape(y_test_lstm.shape)

c:\Users\User\Desktop\test2\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.0347 - val_loss: 0.0039
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0103 - val_loss: 0.0024
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0082 - val_loss: 0.0027
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0070 - val_loss: 0.0019
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0056 - val_loss: 0.0035
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0051 - val_loss: 0.0023
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0047 - val_loss: 0.0025
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0042 - val_loss: 0.0019
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0042 - val_loss: 0.0013
Epoch 10/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0039 - val_loss: 0.0013
Epoch 11/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0037 - val_loss: 0.0013
Epoch 12/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0

In [6]:
xgb_models = []
y_preds_xgb = []
xgb_features_idx = [(raw_features + engineered_features).index(f) for f in engineered_features ]

X_train_xgb = X_train[:,xgb_features_idx]
X_val_xgb = X_val[:,xgb_features_idx]
X_test_xgb = X_test[:,xgb_features_idx]

params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": 0.1,
    "max_depth": 5
}


for step in range(forecast_horizon):
    # target is shifted for each horizon step
    y_train_step = y_train[lookback + step: len(y_train) - forecast_horizon + step + 1]
    y_val_step   = y_val[lookback + step: len(y_val) - forecast_horizon + step + 1]
    y_test_step  = y_test[lookback + step: len(y_test) - forecast_horizon + step + 1]
    
    dtrain = xgb.DMatrix(X_train_xgb[lookback:len(y_train_step)+lookback], label=y_train_step)
    dval   = xgb.DMatrix(X_val_xgb[lookback:len(y_val_step)+lookback], label=y_val_step)
    dtest  = xgb.DMatrix(X_test_xgb[lookback:len(y_test_step)+lookback])

    model_step = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=[(dtrain, "train"), (dval, "val")],
        early_stopping_rounds=20,
        verbose_eval=False
    )
    xgb_models.append(model_step)
    y_preds_xgb.append(model_step.predict(dtest))

y_pred_xgb = np.column_stack(y_preds_xgb)  # shape: (samples, horizon)
y_pred_xgb_inv = scaler_y.inverse_transform(y_pred_xgb)


In [25]:
y_pred_hybrid_inv = 0.5 * y_pred_lstm_inv + 0.5 * y_pred_xgb_inv


In [16]:
# 1. Compute validation RMSE per horizon
val_errors_lstm = [root_mean_squared_error(y_val_lstm[:,h], y_pred_lstm[:,h])
                   for h in range(forecast_horizon)]
val_errors_xgb  = [root_mean_squared_error(y_val_lstm[:,h], y_pred_xgb[:,h])
                   for h in range(forecast_horizon)]

# 2. Convert to normalized weights (lower error = higher weight)
val_errors_lstm = np.array(val_errors_lstm)
val_errors_xgb  = np.array(val_errors_xgb)
weights_lstm = 1 / val_errors_lstm
weights_xgb  = 1 / val_errors_xgb
weights_sum = weights_lstm + weights_xgb
weights_lstm /= weights_sum
weights_xgb  /= weights_sum

# 3. Combine predictions using weights
y_pred_hybrid = weights_lstm * y_pred_lstm_inv + weights_xgb * y_pred_xgb_inv



ValueError: Found input variables with inconsistent numbers of samples: [365, 136]

In [7]:
# --- 1. Get validation predictions ---
# LSTM validation prediction
y_pred_val_lstm = model.predict(X_val_lstm)
y_pred_val_lstm_inv = scaler_y.inverse_transform(
    y_pred_val_lstm.reshape(-1,1)
).reshape(y_pred_val_lstm.shape)

# XGBoost validation prediction
y_preds_val_xgb = []
for step, model_step in enumerate(xgb_models):
    dval = xgb.DMatrix(X_val_xgb[lookback: len(y_val_lstm) + lookback])
    y_preds_val_xgb.append(model_step.predict(dval))

y_pred_val_xgb = np.column_stack(y_preds_val_xgb)
y_pred_val_xgb_inv = scaler_y.inverse_transform(y_pred_val_xgb)

# --- 2. Compute validation errors per horizon ---
val_errors_lstm = [
    root_mean_squared_error(y_val_lstm[:, h], y_pred_val_lstm_inv[:, h])
    for h in range(forecast_horizon)
]
val_errors_xgb = [
    root_mean_squared_error(y_val_lstm[:, h], y_pred_val_xgb_inv[:, h])
    for h in range(forecast_horizon)
]

# --- 3. Convert errors into weights (lower error = higher weight) ---
val_errors_lstm = np.array(val_errors_lstm)
val_errors_xgb  = np.array(val_errors_xgb)

weights_lstm = 1 / val_errors_lstm
weights_xgb  = 1 / val_errors_xgb
weights_sum  = weights_lstm + weights_xgb
weights_lstm /= weights_sum
weights_xgb  /= weights_sum

print("Validation Weights (per horizon):")
for h in range(forecast_horizon):
    print(f"H{h+1}: LSTM={weights_lstm[h]:.2f}, XGB={weights_xgb[h]:.2f}")

# --- 4. Apply weights on test predictions ---
y_pred_hybrid_inv = weights_lstm * y_pred_lstm_inv + weights_xgb * y_pred_xgb_inv


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Validation Weights (per horizon):
H1: LSTM=0.51, XGB=0.49
H2: LSTM=0.50, XGB=0.50
H3: LSTM=0.50, XGB=0.50
H4: LSTM=0.50, XGB=0.50
H5: LSTM=0.51, XGB=0.49
H6: LSTM=0.50, XGB=0.50
H7: LSTM=0.51, XGB=0.49


In [8]:
for h in range(forecast_horizon):
    rmse = root_mean_squared_error(y_test_lstm_inv[:, h], y_pred_lstm_inv[:, h])
    mae = mean_absolute_error(y_test_lstm_inv[:, h], y_pred_lstm_inv[:, h])
    mape = mean_absolute_percentage_error(y_test_lstm_inv[:, h], y_pred_lstm_inv[:, h])
    r2 = r2_score(y_test_lstm_inv[:, h], y_pred_lstm_inv[:, h])
    print(f"Horizon {h+1}: RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.2%}, R^2={r2:.2%}")


Horizon 1: RMSE=2.7453, MAE=2.2943, MAPE=4.91%, R^2=44.41%
Horizon 2: RMSE=2.8986, MAE=2.3507, MAPE=5.00%, R^2=39.44%
Horizon 3: RMSE=2.2018, MAE=1.7517, MAPE=3.76%, R^2=65.93%
Horizon 4: RMSE=2.1854, MAE=1.7126, MAPE=3.68%, R^2=67.11%
Horizon 5: RMSE=3.2030, MAE=2.6358, MAPE=5.63%, R^2=30.18%
Horizon 6: RMSE=2.4596, MAE=1.9217, MAPE=4.14%, R^2=59.41%
Horizon 7: RMSE=2.8016, MAE=2.1438, MAPE=4.59%, R^2=48.08%


In [9]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

for h in range(forecast_horizon):
    rmse = root_mean_squared_error(y_test_lstm_inv[:, h], y_pred_xgb_inv[:, h])
    mae  = mean_absolute_error(y_test_lstm_inv[:, h], y_pred_xgb_inv[:, h])
    mape = mean_absolute_percentage_error(y_test_lstm_inv[:, h], y_pred_xgb_inv[:, h])
    r2   = r2_score(y_test_lstm_inv[:, h], y_pred_xgb_inv[:, h])
    print(f"Horizon {h+1}: RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.2%}, R²={r2:.2%}")

Horizon 1: RMSE=0.4647, MAE=0.3564, MAPE=0.76%, R²=98.41%
Horizon 2: RMSE=0.8813, MAE=0.7243, MAPE=1.59%, R²=94.40%
Horizon 3: RMSE=1.2142, MAE=0.9919, MAPE=2.18%, R²=89.64%
Horizon 4: RMSE=1.4142, MAE=1.1020, MAPE=2.46%, R²=86.23%
Horizon 5: RMSE=1.6997, MAE=1.3338, MAPE=2.94%, R²=80.34%
Horizon 6: RMSE=1.7817, MAE=1.3618, MAPE=3.02%, R²=78.70%
Horizon 7: RMSE=1.9920, MAE=1.4474, MAPE=3.21%, R²=73.75%


In [10]:
for h in range(forecast_horizon):
    rmse = root_mean_squared_error(y_test_lstm_inv[:, h], y_pred_hybrid_inv[:, h])
    mae  = mean_absolute_error(y_test_lstm_inv[:, h], y_pred_hybrid_inv[:, h])
    mape = mean_absolute_percentage_error(y_test_lstm_inv[:, h], y_pred_hybrid_inv[:, h])
    r2   = r2_score(y_test_lstm_inv[:, h], y_pred_hybrid_inv[:, h])
    
    print(f"Horizon {h+1}: RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.2%}, R²={r2:.2%}")

Horizon 1: RMSE=1.2886, MAE=1.0560, MAPE=2.26%, R²=87.75%
Horizon 2: RMSE=1.3437, MAE=1.0886, MAPE=2.33%, R²=86.99%
Horizon 3: RMSE=1.0887, MAE=0.8830, MAPE=1.91%, R²=91.67%
Horizon 4: RMSE=1.3269, MAE=1.0839, MAPE=2.35%, R²=87.88%
Horizon 5: RMSE=1.6621, MAE=1.3169, MAPE=2.83%, R²=81.20%
Horizon 6: RMSE=1.4714, MAE=1.1883, MAPE=2.58%, R²=85.48%
Horizon 7: RMSE=1.7256, MAE=1.3515, MAPE=2.91%, R²=80.30%


In [11]:
import matplotlib.pyplot as plt
import numpy as np

# Choose a starting index in test set to plot (e.g. first forecast window)
start_idx = 0

# Actual data for the forecast horizon
actual = y_test_lstm_inv[start_idx]

# Predictions from each model
pred_lstm   = y_pred_lstm_inv[start_idx]
pred_xgb    = y_pred_xgb_inv[start_idx]
pred_hybrid = y_pred_hybrid[start_idx]

# Build x-axis (Day 1 ... Day 7 relative to forecast start)
days = np.arange(1, forecast_horizon + 1)

plt.figure(figsize=(10, 5))
plt.plot(days, actual, label="Actual", marker="o", linewidth=2, color="black")
plt.plot(days, pred_lstm, label="LSTM", marker="o", linestyle="--")
plt.plot(days, pred_xgb, label="XGBoost", marker="o", linestyle="--")
plt.plot(days, pred_hybrid, label="Hybrid", marker="o", linestyle="--")

plt.title(f"Forecast vs Actual (Test Start Index: {start_idx})")
plt.xlabel("Forecast Horizon (Days Ahead)")
plt.ylabel("FFB OER (%)")
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'y_pred_hybrid' is not defined

In [13]:
# --- Recommendation System Layer ---

# Use last actual price from test set as "current" reference
current_price = y_test_lstm_inv[-1, 0]  

# Hybrid 7-day forecast (latest window)
latest_forecast = y_pred_hybrid_inv[-1]  # shape: (7,)

# Compute average and % change
avg_forecast = np.mean(latest_forecast)
pct_change = ((avg_forecast - current_price) / current_price) * 100

# Recommendation logic
threshold = 2.0  # 2% threshold, tune this value
if pct_change > threshold:
    recommendation = f"HOLD/WAIT (Price expected to rise by {pct_change:.2f}%)"
elif pct_change < -threshold:
    recommendation = f"SELL NOW (Price expected to drop by {pct_change:.2f}%)"
else:
    recommendation = f"NEUTRAL (Price change small: {pct_change:.2f}%)"

print("Current Price:", current_price)
print("7-Day Forecast:", latest_forecast)
print("Average Forecast:", avg_forecast)
print("Recommendation:", recommendation)


Current Price: 38.58
7-Day Forecast: [38.02784017 38.4066775  38.55591555 38.66990518 38.3741187  38.73831237
 38.63097242]
Average Forecast: 38.486248841849864
Recommendation: NEUTRAL (Price change small: -0.24%)
